<a href="https://colab.research.google.com/github/sayakpaul/BERT-for-Mobile/blob/master/Evaluation_IMDB_DistilBERT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!wget https://github.com/sayakpaul/BERT-for-Mobile/releases/download/v0.1.0/IMDB_Models.zip
!unzip -qq IMDB_Models.zip

In [ ]:
!pip install transformers

In [1]:
import tensorflow as tf
print(tf.__version__)

2.3.0


In [5]:
import os
import numpy as np
import pandas as pd
import transformers

from sklearn.metrics import accuracy_score

# fix random seed for reproducibility
seed = 42
np.random.seed(seed)
tf.random.set_seed(seed)

In [6]:
dataset = pd.read_csv(r'https://github.com/dipanjanS/nlp_workshop_dhs18/raw/master/Unit%2011%20-%20Sentiment%20Analysis%20-%20Unsupervised%20Learning/movie_reviews.csv.bz2', compression='bz2')
dataset['sentiment'] = [1 if record == 'positive' else 0 for record in dataset['sentiment']]
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   review     50000 non-null  object
 1   sentiment  50000 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 781.4+ KB


In [7]:
reviews = dataset['review'].values
sentiments = dataset['sentiment'].values

# Taking a small portion to save time
val_reviews = reviews [5000:5900]
val_sentiments = sentiments [5000:5900]

print(val_reviews.shape)

(900,)


In [8]:
tokenizer = transformers.DistilBertTokenizer.from_pretrained('distilbert-base-uncased')

In [9]:
import tqdm

def create_bert_input_features(tokenizer, docs, max_seq_length):
    
    all_ids, all_masks = [], []
    for doc in tqdm.tqdm(docs, desc="Converting docs to features"):
        tokens = tokenizer.tokenize(doc)
        if len(tokens) > max_seq_length-2:
            tokens = tokens[0 : (max_seq_length-2)]
        tokens = ['[CLS]'] + tokens + ['[SEP]']
        ids = tokenizer.convert_tokens_to_ids(tokens)
        masks = [1] * len(ids)
        # Zero-pad up to the sequence length.
        while len(ids) < max_seq_length:
            ids.append(0)
            masks.append(0)
        all_ids.append(ids)
        all_masks.append(masks)
    encoded = np.array([all_ids, all_masks])
    return encoded

In [10]:
MAX_SEQ_LENGTH = 500

val_features_ids, val_features_masks = create_bert_input_features(tokenizer, val_reviews, 
                                                                  max_seq_length=MAX_SEQ_LENGTH)
print('Val Features:', val_features_ids.shape, val_features_masks.shape)

Converting docs to features: 100%|██████████| 900/900 [00:04<00:00, 223.39it/s]


Val Features: (900, 500) (900, 500)


In [11]:
interpreter = tf.lite.Interpreter(model_path="distilbert_imdb_tflite.tflite")
interpreter.allocate_tensors()
interpreter.get_input_details()

[{'dtype': numpy.int32,
  'index': 0,
  'name': 'bert_input_ids',
  'quantization': (0.0, 0),
  'quantization_parameters': {'quantized_dimension': 0,
   'scales': array([], dtype=float32),
   'zero_points': array([], dtype=int32)},
  'shape': array([  1, 500], dtype=int32),
  'shape_signature': array([ -1, 500], dtype=int32),
  'sparsity_parameters': {}},
 {'dtype': numpy.int32,
  'index': 1,
  'name': 'bert_input_masks',
  'quantization': (0.0, 0),
  'quantization_parameters': {'quantized_dimension': 0,
   'scales': array([], dtype=float32),
   'zero_points': array([], dtype=int32)},
  'shape': array([  1, 500], dtype=int32),
  'shape_signature': array([ -1, 500], dtype=int32),
  'sparsity_parameters': {}}]

In [12]:
sample_input_index = np.expand_dims(val_features_ids[0], 0).astype(np.int32)
sample_masks_index = np.expand_dims(val_features_masks[0], 0).astype(np.int32)

In [13]:
bert_input_index = interpreter.get_input_details()[0]["index"]
bert_input_masks_index = interpreter.get_input_details()[1]["index"]
output_index = interpreter.get_output_details()[0]["index"]

interpreter.set_tensor(bert_input_index, sample_input_index)
interpreter.set_tensor(bert_input_masks_index, sample_masks_index)
interpreter.invoke()
prediction = interpreter.tensor(output_index)
print(prediction()[0]) # if prediction > 0.5 then class = 1 else class = 0

[0.7358074]


In [14]:
# Let's convert the data type of the validation features from int64 to int32
val_features_ids = val_features_ids.astype(np.int32)
val_features_masks = val_features_masks.astype(np.int32)

In [15]:
def get_predictions(interpreter):
    predictions = []
    bert_input_index = interpreter.get_input_details()[0]["index"]
    bert_input_masks_index = interpreter.get_input_details()[1]["index"]
    output_index = interpreter.get_output_details()[0]["index"]

    for (i, feature) in enumerate(tqdm.tqdm(val_features_ids)):
        # Prepare the inputs
        feature = np.expand_dims(feature, 0)
        mask = np.expand_dims(val_features_masks[i], 0)

        # Run inference
        interpreter.set_tensor(bert_input_index, sample_input_index)
        interpreter.set_tensor(bert_input_masks_index, sample_masks_index)
        interpreter.invoke()

        # Parse the prediction
        prediction = interpreter.tensor(output_index)()[0]
        if prediction > 0.5:
            prediction = 1
        else:
            prediction = 0
        
        predictions.append(prediction)

    return predictions

In [17]:
interpreter = tf.lite.Interpreter(model_path="distilbert_imdb_tflite.tflite")
interpreter.allocate_tensors()
predictions = get_predictions(interpreter)
accuracy = accuracy_score(val_sentiments, predictions)
print("Accuracy: {}%".format(accuracy * 100.))

Accuracy: 51.0%


In [18]:
interpreter = tf.lite.Interpreter(model_path="distilbert_imdb_tflite_fp16.tflite")
interpreter.allocate_tensors()
predictions = get_predictions(interpreter)
accuracy = accuracy_score(val_sentiments, predictions)
print("Accuracy: {}%".format(accuracy * 100.))

100%|██████████| 900/900 [17:43<00:00,  1.18s/it]

Accuracy: 49.0%
